In [1]:
import warnings
import os
import numpy as np
import pandas as pd
import datetime 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
data_path = "VN_housing_dataset.csv" 
df = pd.read_csv(data_path)
df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 4: invalid continuation byte

In [3]:
print("The initial length of the dataset is", str(len(df)), "rows.")

NameError: name 'df' is not defined

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82497 entries, 0 to 82496
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       82496 non-null  float64
 1   Ngày             82496 non-null  object 
 2   Địa chỉ          82449 non-null  object 
 3   Quận             82495 non-null  object 
 4   Huyện            82449 non-null  object 
 5   Loại hình nhà ở  82465 non-null  object 
 6   Giấy tờ pháp lý  53610 non-null  object 
 7   Số tầng          36399 non-null  object 
 8   Số phòng ngủ     82458 non-null  object 
 9   Diện tích        82495 non-null  object 
 10  Dài              19827 non-null  object 
 11  Rộng             35445 non-null  object 
 12  Giá/m2           82484 non-null  object 
dtypes: float64(1), object(12)
memory usage: 8.2+ MB


In [28]:
#đổi tên cột
df_renamed = df.rename(columns = {"Ngày":"date", "Địa chỉ":"address", "Quận":"district",
                                  "Huyện":"ward", "Loại hình nhà ở":"type_of_housing",
                                 "Giấy tờ pháp lý":"legal_paper", "Số tầng":"num_floors",
                                 "Số phòng ngủ":"num_bed_rooms", "Diện tích":"squared_meter_area",
                                 "Dài":"length_meter", "Rộng":"width_meter", "Giá/m2":"price_in_million_per_square_meter"})
#xóa cột k cần thiết
df_renamed = df_renamed.drop("Unnamed: 0", axis = 1)
# loại bỏ các giá trị thiếu
df_renamed = df_renamed.dropna()
#đặt lại chỉ mục
df_renamed = df_renamed.reset_index()

#in độ dài của tập dữ liệu
print("The length of the dataset after dropping null values is", str(len(df_renamed)), "rows.")

The length of the dataset after dropping null values is 11473 rows.


In [29]:
#Loại bỏ các dòng có số tầng là "Nhiều hơn 10" và số phòng ngủ không xác định:
df_renamed = df_renamed[df_renamed['num_floors'] != 'Nhiều hơn 10']
df_renamed = df_renamed[df_renamed['num_bed_rooms'] != 'nhiều hơn 10 phòng']

# Chỉnh sửa và chuyển đổi kiểu dữ liệu của một số cột:
df_renamed['district'] = df_renamed['district'].str.replace('Quận ','').str.strip()
df_renamed['ward'] = df_renamed['ward'].str.replace('Phường ','').str.strip()
df_renamed['num_floors'] = df_renamed['num_floors'].str.strip().astype(float)
df_renamed['num_bed_rooms'] = df_renamed['num_bed_rooms'].str.replace(' phòng','').str.strip().astype(float)
df_renamed['squared_meter_area'] = df_renamed['squared_meter_area'].str.replace(' m²','').str.strip().astype(float)
df_renamed['length_meter'] = df_renamed['length_meter'].str.replace(' m','').str.strip().astype(float)
df_renamed['width_meter'] = df_renamed['width_meter'].str.replace(' m','').str.strip().astype(float)

# Chuyển đổi giá cả về đơn vị triệu/m²
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²'), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²'), 'price_in_million_per_square_meter'].str.replace(' tỷ/m²','').str.replace('.','').str.replace(',','.').astype(float) * 1000
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'].str.replace(' triệu/m²','').str.replace(',','.').astype(float)
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'].str.replace(' triệu/m²','').str.replace(',','.').astype(float)
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'].str.replace(' đ/m²','').str.replace('.','').astype(float) * 0.000001
print("The number of row after cleaning data:",len(df_renamed))

The number of row after cleaning data: 11313


trực quan hóa dữ liệu

In [30]:
# Create dummies for categorical columns
#tạo biến gỉa cho cột type of housing
dummy_type_of_housing = pd.get_dummies(df_renamed.type_of_housing, prefix="housing_type")
# tạo biến giả cho cột legal_paper
dummy_legal_paper = pd.get_dummies(df_renamed.legal_paper, prefix="legal_paper")
dummy_district = pd.get_dummies(df_renamed.district, prefix="district")
dummy_ward = pd.get_dummies(df_renamed.ward, prefix="ward")
#kết hợp các biến giả với dataframe
df_cleaned = pd.concat([df_renamed, dummy_type_of_housing, dummy_legal_paper, dummy_district, dummy_ward], axis=1)
#loại bỏ các cột không cần thiết
df_cleaned = df_cleaned.drop(['index', 'date', 'address', 'district', 'ward', 'type_of_housing', 'legal_paper'], axis = 1)
df_cleaned.head()

,num_floors,num_bed_rooms,squared_meter_area,length_meter,width_meter,price_in_million_per_square_meter,housing_type_Nhà biệt thự,"housing_type_Nhà mặt phố, mặt tiền","housing_type_Nhà ngõ, hẻm",housing_type_Nhà phố liền kề,...,ward_Đại Mỗ,ward_Định Công,ward_Đống Mác,ward_Đồng Mai,ward_Đồng Nhân,ward_Đồng Tâm,ward_Đồng Xuân,ward_Đội Cấn,ward_Đức Giang,ward_Đức Thắng
0,4.0,4.0,40.0,10.0,4.0,65.0,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,5.0,4.0,52.0,12.0,4.2,93.27,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,5.0,5.0,90.0,18.0,5.0,108.89,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4.0,3.0,32.0,6.6,4.5,60.94,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,3.0,2.0,42.0,11.0,4.0,29.76,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
def remove_outlier_IQR(df, series):
    Q1=df[series].quantile(0.25)
    Q3=df[series].quantile(0.75)
    IQR=Q3-Q1
    df_final=df[~((df[series]<(Q1-1.5*IQR)) | (df[series]>(Q3+1.5*IQR)))]
    return df_final

removed_outliers = df_cleaned
columns_to_remove_outliers = ['num_floors', 'num_bed_rooms', 'squared_meter_area', 'length_meter',
                              'width_meter', 'price_in_million_per_square_meter']
for column in columns_to_remove_outliers:
    removed_outliers = remove_outlier_IQR(removed_outliers, column)
    
print("The final length of the dataset is", str(len(removed_outliers)), "rows.")

The final length of the dataset is 7328 rows.


In [23]:
# Loại bỏ các giá trị ngoại lệ khỏi dữ liệu nhà
housing = removed_outliers

# Tách biến dự đoán và biến phản ứng (giá) 
X = housing.loc[:, housing.columns != 'price_in_million_per_square_meter']
y = housing[['price_in_million_per_square_meter']]
to_be_scaled = ['num_floors', 'num_bed_rooms', 'squared_meter_area', 'length_meter', 'width_meter']

# Khởi tạo bộ điều chuẩn
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

X_scaled = X
y_scaled = y

# Lưu trữ đối tượng fit để tham chiếu và đảo ngược quá trình chuẩn hóa sau này
PredictorScalerFit = PredictorScaler.fit(X_scaled[to_be_scaled])
TargetVarScalerFit = TargetVarScaler.fit(y_scaled)

# Tạo giá trị đã được chuẩn hóa của X và y
X_scaled[to_be_scaled] = PredictorScalerFit.transform(X_scaled[to_be_scaled])
y_scaled = TargetVarScalerFit.transform(y)

X_array = np.array(X_scaled.values).astype("float32")
y_array = np.array(y_scaled).astype("float32")

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=2032)

# Kiểm tra tính đúng đắn của kích thước các tập huấn luyện và kiểm tra
if X_train.shape[0] == y_train.shape[0] and X_train.shape[1] == X_test.shape[1] and X_test.shape[0] == y_test.shape[0] and y_train.shape[1] == y_test.shape[1]:
    print("Tất cả các tập huấn luyện và kiểm tra có kích thước đúng.")


C:\Users\ducan\AppData\Local\Temp\ipykernel_25488\1172724163.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_scaled[to_be_scaled]=PredictorScalerFit.transform(X_scaled[to_be_scaled])


All train and test sets have correct dimensions.


In [24]:
#  các tham số ngẫu nhiên cho mô hình RandomForest
RF_random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [int(x) for x in np.linspace(10, 100, num = 10)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

In [25]:
#Tắt thông báo và cảnh báo của TensorFlow:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_SETTINGS"] = "false"

#Tạo mô hình cơ bản RandomForestRegressor và thực hiện tìm kiếm ngẫu nhiên:
RF_regressor = RandomForestRegressor()
RF_random_search = RandomizedSearchCV(estimator=RF_regressor, param_distributions=RF_random_grid, n_iter=100, cv=5, 
                                      verbose=0, random_state=2022, n_jobs = -1).fit(X_train, np.ravel(y_train))
RF_best_params = RF_random_search.best_params_
RF_best_params

C:\Users\ducan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
135 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
47 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ducan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ducan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-pa

{'n_estimators': 700,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 90,
 'bootstrap': True}

In [27]:
RF_param_grid = {'n_estimators': [RF_best_params['n_estimators']-100, RF_best_params['n_estimators'], RF_best_params['n_estimators']+100],
               'max_features': ['sqrt', 'log2'],
               'max_depth': [RF_best_params['max_depth'] - 10, RF_best_params['max_depth'], RF_best_params['max_depth']+10],
               'min_samples_split': [5, 10],
               'min_samples_leaf': [1, 2],
               'bootstrap': [True, False]}

In [28]:
# Turn off TensorFlow messages and warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_SETTINGS"] = "false"

# Create another base RF model and fit the grid search
RF_regressor_2 = RandomForestRegressor()
RF_grid_search = GridSearchCV(estimator=RF_regressor_2, param_grid=RF_param_grid, 
                              cv=3, n_jobs=-1, verbose=0).fit(X_train, np.ravel(y_train))

# Showing the best parameters
RF_grid_search.best_params_

{'bootstrap': False,
 'max_depth': 80,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 800}


Training the RF model with the best parameters

In [26]:
# Tạo dự đoán trên dữ liệu kiểm tra
RF_predictions = RF.predict(X_test)

# Chuyển đổi dự đoán thành mảng 2D nếu cần thiết
RF_predictions = RF_predictions.reshape(-1, 1)

# Đưa dữ liệu giá dự đoán về tỷ lệ giá ban đầu
RF_predictions = TargetVarScalerFit.inverse_transform(RF_predictions)

#  giá thực tế để so sánh
actual_prices = TargetVarScalerFit.inverse_transform(y_test)  # Thay thế y_test bằng biến mục tiêu thực tees

# Tạo DataFrame để hiển thị giá thực và giá dự đoán
result_df = pd.DataFrame({
    'Giá_Thực': actual_prices.flatten(),
    'Giá_Dự_Đoán_RF': RF_predictions.flatten()
})

# Hiển thị kết quả
print(result_df.head())


   Actual_Price  Predicted_Price_RF
0     77.139999           73.851613
1    121.949997          120.051339
2     66.669998           74.834780
3     80.559998           90.667676
4    133.330002          109.351878


In [1]:
# Định nghĩa hàm tính độ chính xác dựa trên MAPE
def Accuracy_Score(orig, pred):
    MAPE = np.mean(100 * (np.abs(orig - pred) / orig))
    return(100-MAPE)
# Tính độ chính xác của mô hình RF
accuracy = Accuracy_Score(actual_prices, RF_predictions)
print(f"Accuracy for the RF model is: {accuracy:.2f}%")

NameError: name 'actual_prices' is not defined